Importación Librerias 

In [1]:
import pandas as pd
import numpy as np

Importación funciones

Carga de datos 

In [2]:
# Cargar los archivos CSV
box_score_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_box_score.csv')
comparison_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_comparison.csv')
header_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_header.csv')

players_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_players.csv')
points_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_points.csv')
teams_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data/euroleague_teams.csv')


Filtrado por temporada  2023 

In [3]:
# Filtrar por temporada 2023
box_score_2023 = box_score_df[box_score_df['season_code'] == 'E2023']
comparison_2023 = comparison_df[comparison_df['season_code'] == 'E2023']
header_2023 = header_df[header_df['season_code'] == 'E2023']

players_2023 = players_df[players_df['season_code'] == 'E2023']
points_2023 = points_df[points_df['season_code'] == 'E2023']
teams_2023 = teams_df[teams_df['season_code'] == 'E2023']


Creación csv

In [4]:
# Crear archivos CSV para los datos filtrados por temporada 2023
box_score_2023.to_csv('box_score_2023.csv', index=False)
comparison_2023.to_csv('comparison_2023.csv', index=False)
header_2023.to_csv('header_2023.csv', index=False)

players_2023.to_csv('players_2023.csv', index=False)
points_2023.to_csv('points_2023.csv', index=False)
teams_2023.to_csv('teams_2023.csv', index=False)

print("Archivos CSV creados correctamente.")


Archivos CSV creados correctamente.


Creacion de nuevas variables para comparar rendimiento 

Diferencia de puntos 

In [4]:
# Calcular la diferencia de puntos para cada partido
header_df['point_diff_a'] = header_df['score_a'] - header_df['score_b']
header_df['point_diff_b'] = header_df['score_b'] - header_df['score_a']

# Calcular la diferencia de puntos promedio para cada equipo
point_diff_avg = header_df.groupby('team_id_a')['point_diff_a'].mean().reset_index().rename(columns={'point_diff_a': 'point_diff_avg_a'})
point_diff_avg_b = header_df.groupby('team_id_b')['point_diff_b'].mean().reset_index().rename(columns={'point_diff_b': 'point_diff_avg_b'})

# Unir las diferencias de ambos equipos
point_diff_avg = point_diff_avg.merge(point_diff_avg_b, left_on='team_id_a', right_on='team_id_b', how='outer')


Ratio victorias/derrotas

In [5]:
# Calcular victorias y derrotas para cada equipo
header_df['team_a_win'] = header_df['score_a'] > header_df['score_b']
header_df['team_b_win'] = header_df['score_b'] > header_df['score_a']

# Ratio de victorias/derrotas por equipo
wins_a = header_df.groupby('team_id_a')['team_a_win'].sum().reset_index().rename(columns={'team_a_win': 'wins_a'})
losses_a = header_df.groupby('team_id_a')['team_a_win'].count().reset_index().rename(columns={'team_a_win': 'total_games_a'})
win_loss_ratio_a = wins_a.merge(losses_a, on='team_id_a')
win_loss_ratio_a['win_loss_ratio_a'] = win_loss_ratio_a['wins_a'] / (win_loss_ratio_a['total_games_a'] - win_loss_ratio_a['wins_a'])

wins_b = header_df.groupby('team_id_b')['team_b_win'].sum().reset_index().rename(columns={'team_b_win': 'wins_b'})
losses_b = header_df.groupby('team_id_b')['team_b_win'].count().reset_index().rename(columns={'team_b_win': 'total_games_b'})
win_loss_ratio_b = wins_b.merge(losses_b, on='team_id_b')
win_loss_ratio_b['win_loss_ratio_b'] = win_loss_ratio_b['wins_b'] / (win_loss_ratio_b['total_games_b'] - win_loss_ratio_b['wins_b'])


Eficiencia de ataque y defensa

In [7]:
from Funciones import *


                     player  points  possessions  offensive_efficiency  \
0               SCOTT, MIKE      10         8.00              1.250000   
1                LEE, PARIS      17        17.76              0.957207   
2           KAHUDI, CHARLES       5         3.00              1.666667   
3        LAUVERGNE, JOFFREY      16        14.00              1.142857   
4  LUWAWU-CABARROT, TIMOTHE      20        15.20              1.315789   

   defensive_efficiency  
0              0.800000  
1              1.044706  
2              0.600000  
3              0.875000  
4              0.760000  


Rendimiento en casa vs. fuera

In [6]:
# Rendimiento en casa
header_df['is_home'] = header_df['team_a'] == header_df['stadium']

# Calcular puntos promedio en casa y fuera
home_performance = header_df.groupby('team_id_a')['score_a'].mean().reset_index().rename(columns={'score_a': 'home_points_avg'})
away_performance = header_df.groupby('team_id_b')['score_b'].mean().reset_index().rename(columns={'score_b': 'away_points_avg'})

# Unir los resultados de ambos equipos
performance_comparison = home_performance.merge(away_performance, left_on='team_id_a', right_on='team_id_b', how='outer')


Historial de enfrentamientos directos

In [7]:
# Historial de enfrentamientos entre equipos
head_to_head = header_df.groupby(['team_id_a', 'team_id_b']).agg({'team_a_win': 'sum', 'team_b_win': 'sum'}).reset_index()
head_to_head['total_matches'] = head_to_head['team_a_win'] + head_to_head['team_b_win']

# Ver quién ha ganado más veces en enfrentamientos directos
# Convertir la columna 'dominance' a valores binarios (1 y 0)
head_to_head['dominance'] = np.where(head_to_head['team_a_win'] > head_to_head['team_b_win'], 1, 0)

# Verificar los primeros valores de la columna nueva
print(head_to_head[['team_a_win', 'team_b_win', 'dominance']].head())



   team_a_win  team_b_win  dominance
0           0           1          0
1           1           0          1
2           1           0          1
3           1           0          1
4           1           1          0


In [10]:
head_to_head.columns
tipos_columnas = head_to_head.dtypes
tipos_columnas

team_id_a        object
team_id_b        object
team_a_win        int64
team_b_win        int64
total_matches     int64
dominance         int32
dtype: object

Creacion de csv de las nuevas caracteristicas

In [8]:
# Guardar las nuevas características en archivos CSV
point_diff_avg.to_csv('point_diff_avg_2023.csv', index=False)
performance_comparison.to_csv('home_away_performance_2023.csv', index=False)
head_to_head.to_csv('head_to_head_2023.csv', index=False)

print("Características nuevas guardadas en archivos CSV.")


Características nuevas guardadas en archivos CSV.


In [29]:
performance_comparison.head(10)

,team_id_a,home_points_avg,team_id_b,away_points_avg
0,ARI,79.700000,ARI,70.500000
1,ASV,77.597701,ASV,72.400000
2,AVE,74.000000,AVE,76.800000
3,BAM,78.486842,BAM,72.105263
4,BAR,80.953668,BAR,77.387097
5,BAS,82.547009,BAS,78.936170
6,BER,76.699115,BER,76.521739
7,BES,67.500000,BES,65.166667
8,BIL,78.700000,BIL,72.200000
9,BUD,75.866667,BUD,72.800000


In [9]:
head_to_head.head(10)

,team_id_a,team_id_b,team_a_win,team_b_win,total_matches,dominance
0,ARI,BAS,0,1,1,0
1,ARI,CIB,1,0,1,1
2,ARI,IST,1,0,1,1
3,ARI,LEM,1,0,1,1
4,ARI,LIE,1,1,2,0
5,ARI,MAL,1,0,1,1
6,ARI,MIL,0,1,1,0
7,ARI,TEL,0,1,1,0
8,ARI,ULK,0,1,1,0
9,ASV,BAR,2,3,5,0


In [10]:
point_diff_avg.head(10)

,team_id_a,point_diff_avg_a,team_id_b,point_diff_avg_b
0,ARI,-0.400000,ARI,-6.100000
1,ASV,-3.195402,ASV,-9.682353
2,AVE,-3.400000,AVE,-8.600000
3,BAM,0.289474,BAM,-8.223684
4,BAR,8.494208,BAR,2.391129
5,BAS,5.683761,BAS,-3.370213
6,BER,-2.610619,BER,-7.843478
7,BES,-7.500000,BES,-14.250000
8,BIL,8.700000,BIL,-8.700000
9,BUD,-4.400000,BUD,-16.933333


Convertir Columnas

In [11]:
# Convertir las columnas 'team_id_a' y 'team_id_b' a mayúsculas
point_diff_avg['team_id_a'] = point_diff_avg['team_id_a'].str.upper()
point_diff_avg['team_id_b'] = point_diff_avg['team_id_b'].str.upper()

# Ver el resultado
print(point_diff_avg.head())


  team_id_a  point_diff_avg_a team_id_b  point_diff_avg_b
0       ARI         -0.400000       ARI         -6.100000
1       ASV         -3.195402       ASV         -9.682353
2       AVE         -3.400000       AVE         -8.600000
3       BAM          0.289474       BAM         -8.223684
4       BAR          8.494208       BAR          2.391129


Creacion de CSV

In [12]:
header_df = pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/header_2023.csv') 
df_players =  pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/players_2023.csv')
df_eficiencia =  pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/euroleague_players_efficiency.csv')
df_head_to_head =  pd.read_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/head_to_head_2023.csv')


In [34]:

# Asegurarse de que todas las claves están en mayúsculas
df_head_to_head['team_id_a'] = df_head_to_head['team_id_a'].str.upper()
df_head_to_head['team_id_b'] = df_head_to_head['team_id_b'].str.upper()

point_diff_avg['team_id_a'] = point_diff_avg['team_id_a'].str.upper()
point_diff_avg['team_id_b'] = point_diff_avg['team_id_b'].str.upper()

performance_comparison['team_id_a'] = performance_comparison['team_id_a'].str.upper()
performance_comparison['team_id_b'] = performance_comparison['team_id_b'].str.upper()

# Unir el primer DataFrame con point_diff_avg
final_data = df_head_to_head.merge(point_diff_avg, on=['team_id_a', 'team_id_b'], how='outer')

# Unir el resultado con performance_comparison
final_data = final_data.merge(performance_comparison, on=['team_id_a', 'team_id_b'], how='outer')

# Ver el resultado para asegurarse de que no haya nulos
print(final_data.head())


  team_id_a team_id_b  team_a_win  team_b_win  total_matches  dominance  \
0       ARI       ARI         NaN         NaN            NaN        NaN   
1       ARI       BAS         0.0         1.0            1.0        0.0   
2       ARI       CIB         1.0         0.0            1.0        1.0   
3       ARI       IST         1.0         0.0            1.0        1.0   
4       ARI       LEM         1.0         0.0            1.0        1.0   

   point_diff_avg_a  point_diff_avg_b  home_points_avg  away_points_avg  
0              -0.4              -6.1             79.7             70.5  
1               NaN               NaN              NaN              NaN  
2               NaN               NaN              NaN              NaN  
3               NaN               NaN              NaN              NaN  
4               NaN               NaN              NaN              NaN  


Creacion de CSV

In [37]:
final_data.to_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/final_data.csv', index=False)

In [32]:
# Eliminar filas que contengan valores nulos
final_data_clean = final_data.dropna()


In [35]:
# Verificar si quedan valores nulos
print(final_data_clean.isnull().sum())


team_id_a           0
team_id_b           0
team_a_win          0
team_b_win          0
total_matches       0
dominance           0
point_diff_avg_a    0
point_diff_avg_b    0
home_points_avg     0
away_points_avg     0
dtype: int64


In [36]:
final_data_clean.head(10)
final_data_clean.to_csv('c:/Users/Casa/Documents/GitHub/ProjecyEuroleague/data cleaned/final_data.csv', index=False)

In [16]:
# Unir df_head_to_head con point_diff_avg basado en team_id_a y team_id_b
merged_data = df_head_to_head.merge(point_diff_avg, on=['team_id_a', 'team_id_b'], how='right')

# Unir el resultado anterior con performance_comparison usando team_id_a y team_id_b
merged_data = merged_data.merge(performance_comparison, on=['team_id_a', 'team_id_b'], how='right')

# Visualizar el resultado
print(merged_data.head())


  team_id_a team_id_b  team_a_win  team_b_win  total_matches  dominance  \
0       ARI       ARI         NaN         NaN            NaN        NaN   
1       ASV       ASV         NaN         NaN            NaN        NaN   
2       AVE       AVE         NaN         NaN            NaN        NaN   
3       BAM       BAM         NaN         NaN            NaN        NaN   
4       BAR       BAR         NaN         NaN            NaN        NaN   

   point_diff_avg_a  point_diff_avg_b  home_points_avg  away_points_avg  
0         -0.400000         -6.100000        79.700000        70.500000  
1         -3.195402         -9.682353        77.597701        72.400000  
2         -3.400000         -8.600000        74.000000        76.800000  
3          0.289474         -8.223684        78.486842        72.105263  
4          8.494208          2.391129        80.953668        77.387097  
